In [1]:
## Bracketed Paste is disabled to prevent characters after output
## Example: 
# $ echo 'Hii'
# | Hi?2004l

bind 'set enable-bracketed-paste off'

In [2]:
## TEST:  Make sure simple prompt used (e.g., no escapes that might contaminate output)
## PS1="$ "
## TODO: PS1="> "

## NOTE: The Jupyter bash kernel requires that PS1 not be modified as they customize it. 
echo $PS1
echo $TMP

[PEXP\[\]ECT_PROMPT>



In [3]:
# Delete all aliases and function
# TODO: Instead start from pristine environment
unalias -a
alias | wc -l
for f in $(typeset -f | egrep '^\w+'); do unset -f $f; done
typeset -f | egrep '^\w+' | wc -l

0
0


In [4]:
# Setting a temp directory for tests
TMP=/tmp/test-cp-mv

## NOTE: Source it directly from the ./tests directory.
BIN_DIR=$PWD/..
## You will need to run jupyter from that directory.
## source $TEMP_BIN/_dir-aliases.bash
source _dir-aliases.bash
alias | wc -l

8


In [5]:
## NOTE: For reproducability, the directory name needs to be fixed
## In place of $$, use a psuedo random number (e,g., 1210)
## *** All output from one run to the next needs to be the same ***

# Test safe symbolic links
## temp_dir=$TMP/test-$$
temp_dir=$TMP/test-1210

mkdir -p "$temp_dir"
# TODO: /bin/rm -rvf "$temp_dir"
cd "$temp_dir"
pwd

#ALIAS FOR PRINTING SEPERATION LINES (FOR JUPYTER)
alias linebr="printf '%*s\n' "${COLUMNS:-$(tput cols)}" '' | tr ' ' -"

/tmp/test-cp-mv/test-1210


In [6]:
# Count aliases proper
alias | wc -l

9


In [7]:
# Count functions
typeset -f | egrep '^\w+' | wc -l

2


In [8]:
# Command overrides for moving and copying files
# NOTE: -p option of cp (i.e., --preserve  "preserve file attributes if possible")
# leads to problems when copying files owner by others (although group writable)
#    cp: preserving times for /usr/local/httpd/internal/cgi-bin/phone-list: Operation not permitted
# - other options for cp, mv, and rm: -i interactive; and -v verbose.
other_file_args="-v"

In [9]:
if [ "$OSTYPE" = "solaris" ]; then other_file_args=""; fi

In [10]:
## OLD: alias cls='clear'
## NOTE: Unfortunately clear clobbers the terminal scrollback buffer.
## via https://askubuntu.com/questions/792453/how-to-stop-clear-from-clearing-scrollback-buffer:
##    type CTRL+L instead of clear
## TAKE1: alias cls="printf '\33[H\33[2J'"
##   where \33 is octal code for Escape (i.e., 0x1B)
## TAKE2
alias clear="echo 'use cls instead (or /bin/clear)'"
alias cls="command clear -x"

In [11]:
#Shows that cls must be used instead of clear
clear

use cls instead (or /bin/clear)


In [12]:
#Shows that cls is the new method of clearing screen
alias | grep cls

alias clear='echo '\''use cls instead (or /bin/clear)'\'''
alias cls='command clear -x'


In [13]:
{
    # TODO: see if this is a shellcheck bug
    #    SC2034: MV appears unused. Verify it or export it.
    # shellcheck disable=SC2034
    MV="/bin/mv -i $other_file_args"
}

In [14]:
#All move aliases
alias mv='$MV'
alias move='mv'
alias move-force='move -f'

In [15]:
# TODO: make sure symbolic links are copied as-is (ie, not dereferenced)
CP="/bin/cp -ip $other_file_args"
##reference-variable "$CP"

In [16]:
#All Copy Aliases
alias copy='$CP'
alias copy-force='/bin/cp -fp $other_file_args'
alias cp='/bin/cp -i $other_file_args'

In [17]:
#All Remove Aliases
alias rm='/bin/rm -i $other_file_args'
alias delete='/bin/rm -i $other_file_args'
alias del="delete"
alias delete-force='/bin/rm -f $other_file_args'
alias remove-force='delete-force'

In [18]:
#All remove-dir aliases
alias remove-dir='/bin/rm -rv'
alias delete-dir='remove-dir'
alias remove-dir-force='/bin/rm -rfv'
alias delete-dir-force='remove-dir-force'

In [19]:
#Aliases set for copy and readonly
alias copy-readonly='copy-readonly.sh'
function copy-readonly-spec () {
    local spec="$1"
    local dir="$2"
    if [[ ("$3" != "") || ($dir = "") || ($spec == "") ]]; then
        echo "Usage: copy-readonly-spec pattern dir";
        return
    fi
    # effing shellcheck (SC2086: Double quote to prevent globbing)
    # shellcheck disable=SC2086
    for f in $($LS $spec); do copy-readonly "$f" "$dir"; done
}
# copy-readonly-to-dir(dir, file, ...): variant of copy-readonly-spec with
# directory first and files given in args 2, 3, etc.
function copy-readonly-to-dir () {
    local dir="$1"
    shift
    for f in "$@"; do copy-readonly "$f" "$dir"; done
}

In [20]:
#Exporting NICE and TIME_CMD
export NICE="nice -19"
export TIME_CMD="/usr/bin/time"

In [21]:
#Fixing Dir Permissions
alias fix-dir-permissions="find . -type d -exec chmod go+xs {} \;"

In [22]:
linebr

--------------------------------------------------------------------------------


In [23]:
## MOVE INTO $TMP FOR TESTING
pwd

/tmp/test-cp-mv/test-1210


In [24]:
# TEST 1 - MOVE ALIASES

## CLEARS TEST FOLDER
rm -rf ./*
linebr

## CREATES TEST SUBJECTS
touch abc def ghi
mkdir mvtest_dir1

## WORK OF ALIASES
mv abc ./mvtest_dir1
move def ./mvtest_dir1
move-force ghi ./mvtest_dir1
linebr

## VIEWING THE CHANGES MADE
ls -l
linebr
ls -l ./mvtest_dir1
linebr

removed './abc'
removed './def'
removed './ghi'
--------------------------------------------------------------------------------
renamed 'abc' -> './mvtest_dir1/abc'
renamed 'def' -> './mvtest_dir1/def'
renamed 'ghi' -> './mvtest_dir1/ghi'
--------------------------------------------------------------------------------
total 4
drwxrwxr-x 2 aveey aveey 4096 Jul  3 21:22 mvtest_dir1
--------------------------------------------------------------------------------
total 0
-rw-rw-r-- 1 aveey aveey 0 Jul  3 21:22 abc
-rw-rw-r-- 1 aveey aveey 0 Jul  3 21:22 def
-rw-rw-r-- 1 aveey aveey 0 Jul  3 21:22 ghi
--------------------------------------------------------------------------------


In [25]:
# TEST 2 - COPY ALIASES

## CLEARS TEST FOLDER
rm -rf ./*
linebr

## CREATES TEST SUBJECTS
touch abc def ghi
mkdir cptest_dir1

## WORK OF ALIASES
cp abc ./cptest_dir1
copy def ./cptest_dir1
copy-force ghi ./cptest_dir1
linebr

## VIEWING THE CHANGES MADE
ls -l
linebr
ls -l ./cptest_dir1
linebr

removed './mvtest_dir1/abc'
removed './mvtest_dir1/ghi'
removed './mvtest_dir1/def'
removed directory './mvtest_dir1'
--------------------------------------------------------------------------------
'abc' -> './cptest_dir1/abc'
'def' -> './cptest_dir1/def'
'ghi' -> './cptest_dir1/ghi'
--------------------------------------------------------------------------------
total 4
-rw-rw-r-- 1 aveey aveey    0 Jul  3 21:22 abc
drwxrwxr-x 2 aveey aveey 4096 Jul  3 21:22 cptest_dir1
-rw-rw-r-- 1 aveey aveey    0 Jul  3 21:22 def
-rw-rw-r-- 1 aveey aveey    0 Jul  3 21:22 ghi
--------------------------------------------------------------------------------
total 0
-rw-rw-r-- 1 aveey aveey 0 Jul  3 21:22 abc
-rw-rw-r-- 1 aveey aveey 0 Jul  3 21:22 def
-rw-rw-r-- 1 aveey aveey 0 Jul  3 21:22 ghi
--------------------------------------------------------------------------------


In [26]:
# #=================(TEST ERROR: "/bin/rm: remove regular empty file 'f1'?")=========================#
# # TEST 3 - REMOVE/DELETE ALIASES

# ## CLEARS TEST FOLDER
# rm -rf ./*
# linebr

# ## CREATES TEST SUBJECTS
# touch f1 f2 f3 f4 f5

# ## WORK OF ALIASES
# rm f1
# delete f2
# remove-force f3

# linebr

# ## VIEWING THE CHANGES MADE
# ls -l
# linebr

In [27]:
# TEST 4 - REMOVE DIRECTORY ALIASES

## CLEARS TEST FOLDER
rm -rf ./*
linebr

## CREATES TEST SUBJECTS
touch abc def ghi
mkdir TDIR1 TDIR2 TDIR3 TDIR4
ls -l
linebr
## WORK OF ALIASES
remove-dir "TDIR1"
delete-dir "TDIR2"
remove-dir-force TDIR3
delete-dir-force TDIR4
linebr

## VIEWING THE CHANGES MADE
ls -l
linebr

removed './abc'
removed './cptest_dir1/abc'
removed './cptest_dir1/ghi'
removed './cptest_dir1/def'
removed directory './cptest_dir1'
removed './def'
removed './ghi'
--------------------------------------------------------------------------------
total 16
-rw-rw-r-- 1 aveey aveey    0 Jul  3 21:22 abc
-rw-rw-r-- 1 aveey aveey    0 Jul  3 21:22 def
-rw-rw-r-- 1 aveey aveey    0 Jul  3 21:22 ghi
drwxrwxr-x 2 aveey aveey 4096 Jul  3 21:22 TDIR1
drwxrwxr-x 2 aveey aveey 4096 Jul  3 21:22 TDIR2
drwxrwxr-x 2 aveey aveey 4096 Jul  3 21:22 TDIR3
drwxrwxr-x 2 aveey aveey 4096 Jul  3 21:22 TDIR4
--------------------------------------------------------------------------------
removed directory 'TDIR1'
removed directory 'TDIR2'
removed directory 'TDIR3'
removed directory 'TDIR4'
--------------------------------------------------------------------------------
total 0
-rw-rw-r-- 1 aveey aveey 0 Jul  3 21:22 abc
-rw-rw-r-- 1 aveey aveey 0 Jul  3 21:22 def
-rw-rw-r-- 1 aveey aveey 0 Jul  3 21:22 ghi
-